Table as_ 22_q: Arrivals, and requests for transfer into the UK under the Dublin Regulation, by article and country of transfer

In [1]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 5']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='as_22_q', library='pyexcel-ods3')

In [2]:
df = pd.DataFrame(data.get_array())

In [3]:
Final_table = pd.DataFrame()

In [4]:
observations = df.loc[3:, :15]
observations.rename(columns= observations.iloc[0], inplace=True)
observations.drop(observations.index[0], inplace = True)
new_table = pd.melt(observations,
                       ['Quarter','Country Transferred from '],
                       var_name="Total arrivals into the UK",
                       value_name="Value")
new_table.Value.dropna(inplace =True)
new_table.rename(columns={'Total arrivals into the UK': 'Dublin Regulation Article'}, inplace=True)
new_table['Nature of Transfer'] = 'Arrivals'
new_table['Unit'] = 'Arrivals'
new_table['Measure Type'] = 'Count'
Final_table = pd.concat([Final_table, new_table])

In [5]:
observations1 = df.loc[3:, :30]
observations1.drop(observations1.iloc[: , 2:16], inplace=True, axis=1)
observations1.rename(columns= observations1.iloc[0], inplace=True)
observations1.drop(observations1.index[0], inplace = True)
new_table = pd.melt(observations1,
                       ['Quarter','Country Transferred from '],
                       var_name="Total requests for transfers into the UK",
                       value_name="Value")
new_table.Value.dropna(inplace =True)
new_table.rename(columns={'Total requests for transfers into the UK': 'Dublin Regulation Article'}, inplace=True)
new_table['Nature of Transfer'] = 'Requests for Transfer'
new_table['Unit'] = 'Requests for Transfer'
new_table['Measure Type'] = 'Count'
Final_table = pd.concat([Final_table, new_table])

In [6]:
Final_table['Value'] = Final_table['Value'].map(lambda x : ''
                                                   if (x == '.') | (x == 'z') | ( x == ':')
                                                    else x )

In [7]:
import numpy as np
Final_table['Value'].replace('', np.nan, inplace=True)
Final_table.dropna(subset=['Value'], inplace=True)
Final_table['Value'] = Final_table['Value'].apply(lambda x: pd.to_numeric(x, downcast='integer'))
Final_table['Value'] = Final_table['Value'].astype(int)

In [8]:
Final_table.rename(columns={'Quarter': 'Period'}, inplace=True)
Final_table['Period'] = Final_table['Period'].astype(str)
Final_table = Final_table[Final_table['Period'] != '']
Final_table['Period'] = 'quarter/' + Final_table['Period'].map(lambda cell: cell.replace(' ','-'))

In [9]:
Final_table['Country Transferred'] = Final_table['Country Transferred from '].str.lstrip('*')

In [10]:
import urllib.request as request
import csv
import io
import requests
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_migration/master/codelists/ho-country-of-nationality.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_migration/master/codelists/ho-country-of-nationality.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
Final_table = pd.merge(Final_table, c, how = 'left', left_on = 'Country Transferred', right_on = 'Label')
Final_table.columns = ['Foreign geography' if x=='Notation' else x for x in Final_table.columns]

In [11]:
Final_table = Final_table[['Period','Foreign geography','Nature of Transfer','Dublin Regulation Article','Measure Type','Value','Unit']]

In [12]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.to_csv(destinationFolder / ('as_22_q.csv'), index = False)

In [13]:
Final_table

,Period,Foreign geography,Nature of Transfer,Dublin Regulation Article,Measure Type,Value,Unit
0,quarter/2015-Q1,total,Arrivals,Total arrivals into the UK,Count,41,Arrivals
1,quarter/2015-Q1,austria,Arrivals,Total arrivals into the UK,Count,1,Arrivals
2,quarter/2015-Q1,belgium,Arrivals,Total arrivals into the UK,Count,1,Arrivals
3,quarter/2015-Q1,bulgaria,Arrivals,Total arrivals into the UK,Count,0,Arrivals
4,quarter/2015-Q1,croatia,Arrivals,Total arrivals into the UK,Count,0,Arrivals
5,quarter/2015-Q1,cyprus,Arrivals,Total arrivals into the UK,Count,2,Arrivals
6,quarter/2015-Q1,czech-republic,Arrivals,Total arrivals into the UK,Count,0,Arrivals
7,quarter/2015-Q1,denmark,Arrivals,Total arrivals into the UK,Count,2,Arrivals
8,quarter/2015-Q1,estonia,Arrivals,Total arrivals into the UK,Count,0,Arrivals
9,quarter/2015-Q1,finland,Arrivals,Total arrivals into the UK,Count,2,Arrivals
